# Set-up SQLite Database

In [1]:
import sqlite3
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.36.0


In [2]:
product = pd.read_csv('../data/product.csv')
product_price = pd.read_csv('../data/product_price.csv')
vendor = pd.read_csv('../data/vendor.csv')
store = pd.read_csv('../data/store.csv')
transaction = pd.read_csv('../data/transaction.csv')

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
store.rename(columns = {'Store Number':'StoreID',
                        'Store Name':'StoreName',
                        'Zip Code':'ZipCode'}, inplace = True)

In [4]:
# remove these columns
product = product.drop(['CategoryCode','CategoryName'], axis=1)

In [5]:
print('product: ', product.columns)
print('product_price: ', product_price.columns)
print('vendor: ', vendor.columns)
print('store: ', store.columns)
print('transaction: ', transaction.columns)

product:  Index(['ProductID', 'ProductDescription', 'CategoryGroup', 'Pack',
       'BottleVolume_ml', 'Proof', 'VendorID'],
      dtype='object')
product_price:  Index(['ProductID', 'Date', 'StateBottleRetail', 'StateBottleCost'], dtype='object')
vendor:  Index(['VendorID', 'VendorName'], dtype='object')
store:  Index(['StoreID', 'StoreName', 'Address', 'City', 'ZipCode', 'County', 'lat',
       'lng', 'max', 'min'],
      dtype='object')
transaction:  Index(['TransactionID', 'Date', 'StoreID', 'ProductID', 'VendorID',
       'BottlesSold', 'VolumeSold_Gallons', 'Sale_Dollars'],
      dtype='object')


In [6]:
# delete database (in folder) if you have added a table previously 
dbconn = sqlite3.connect('../data/IowaLiquorSales.db') # database
cursor = dbconn.cursor() # create a cursor to use to execute sql statements

In [7]:
# update to include final columns for each table
cursor.execute('''CREATE TABLE Product
                ([ProductID] TEXT PRIMARY KEY, 
                [ProductDescription] text, 
                [CategoryGroup] text, 
                [Pack] integer, 
                [BottleVolume_ml] real, 
                [Proof] real, 
                [VendorID] integer)''')

cursor.execute('''CREATE TABLE Price
                ([ProductID] text, 
                [Date] date, 
                [StateBottleRetail] real, 
                [StateBottleCost] real,
                PRIMARY KEY(ProductID, Date, StateBottleRetail))''')

cursor.execute('''CREATE TABLE Vendor
                ([VendorID] INTEGER PRIMARY KEY, 
                [VendorName] text)''')

cursor.execute('''CREATE TABLE Store
                ([StoreID] INTEGER PRIMARY KEY, 
                [StoreName] text, 
                [Address] text, 
                [City] text, 
                [ZipCode] text, 
                [County] text, 
                [lat] real, 
                [lng] real, 
                [max] date, 
                [min] date)''')

cursor.execute('''CREATE TABLE Sales
                ([TransactionID]  TEXT PRIMARY KEY, 
                [Date] date, 
                [StoreID] integer, 
                [ProductID] text, 
                [VendorID] integer, 
                [BottlesSold] integer, 
                [VolumeSold_Gallons] integer, 
                [Sale_Dollars] real)''')

In [8]:
product.to_sql('Product', dbconn, if_exists = 'append', index = False)
product_price.to_sql('Price', dbconn, if_exists = 'append', index = False)
vendor.to_sql('Vendor', dbconn, if_exists = 'append', index = False)
store.to_sql('Store', dbconn, if_exists = 'append', index = False)
transaction.to_sql('Sales', dbconn, if_exists = 'append', index = False)

In [9]:
dbconn.commit()

In [10]:
dbconn.close()